# NumPy Support in Numba

Numba is designed to be used with NumPy and complement its capabilities.  Numba supports:

* Passing NumPy arrays as arguments, including structured dtypes
* Creating compiled ufuncs and generalized ufuncs
* Using a large subset of NumPy functions in nopython mode

In [1]:
import numpy as np
import numba
from numba import jit

## Numba Specialization by Dtype

Numba automatically uses multiple dispatch on compiled functions to allow different specialized implementations of the same function to be used.  Suppose we have a function that clamps values to zero if they are below a particular magnitude:

In [2]:
@jit(nopython=True)
def zero_clamp(x, threshold):
    # assume 1D array.  See later in this notebook for more general function
    out = np.empty_like(x)
    for i in range(out.shape[0]):
        if np.abs(x[i]) > threshold:
            out[i] = x[i]
        else:
            out[i] = 0
    return out        

In [3]:
a_small = np.linspace(0, 1, 50)
zero_clamp(a_small, 0.3)

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.30612245, 0.32653061, 0.34693878, 0.36734694, 0.3877551 ,
       0.40816327, 0.42857143, 0.44897959, 0.46938776, 0.48979592,
       0.51020408, 0.53061224, 0.55102041, 0.57142857, 0.59183673,
       0.6122449 , 0.63265306, 0.65306122, 0.67346939, 0.69387755,
       0.71428571, 0.73469388, 0.75510204, 0.7755102 , 0.79591837,
       0.81632653, 0.83673469, 0.85714286, 0.87755102, 0.89795918,
       0.91836735, 0.93877551, 0.95918367, 0.97959184, 1.        ])

Now let's benchmark some different kinds of array inputs.  We'll try:

* int64
* float32
* float32 with a stride (elements not contiguous in memory)

In [4]:
n = 10000
a_int16 = np.arange(n).astype(np.int16)
a_float32 = np.linspace(0, 1, n, dtype=np.float32)
a_float32_strided = np.linspace(0, 1, 2*n, dtype=np.float32)[::2]  # view of every other element

In [5]:
%timeit zero_clamp(a_int16, 1600)
%timeit zero_clamp(a_float32, 0.3)
%timeit zero_clamp(a_float32_strided, 0.3)

3.78 µs ± 185 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
5.63 µs ± 75.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
9.9 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


We see different performance characteristics for each of these cases, even though they have the same number of input elements.  Numba generated different machine code for each situation, which we can see if we look at the `.signatures` attribute of the compiled function:

In [6]:
zero_clamp.signatures

[(array(float64, 1d, C), float64),
 (array(int16, 1d, C), int64),
 (array(float32, 1d, C), float64),
 (array(float32, 1d, A), float64)]

When printed as strings, Numba array types have the form: `array(dtype, dimensions, layout)`.  The first signature therefore corresponds to a 1D array of float64 with C style layout (row-major order, no gaps between elements).  The next two signatures are similar, but for `int16` and `float32` arrays.  The final signature indicates an "any" layout array, which usually happens when you slice an array, and it no longer has a C or FORTRAN memory layout.

We can compare to a pure NumPy implementation and see the speed improvement that Numba has achieved through a combination of specialization and elimination of temporary arrays:

In [7]:
def np_zero_clamp(x, threshold):
    return np.where(np.abs(x) > threshold, x, 0)

In [8]:
%timeit np_zero_clamp(a_int16, 1600)
%timeit np_zero_clamp(a_float32, 0.3)
%timeit np_zero_clamp(a_float32_strided, 0.3)

11.6 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
15.6 µs ± 640 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
27.6 µs ± 5.54 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Creating Ufuncs

Universal functions, typically called "ufuncs" for short, are functions that broadcast an elementwise operation across input arrays of varying numbers of dimensions.  Most NumPy functions are ufuncs, and Numba makes it easy to compile custom ufuncs using the `@vectorize` decorator.

In [9]:
from numba import vectorize

In [10]:
@vectorize(nopython=True)
def ufunc_zero_clamp(x, threshold):
    if np.abs(x) > threshold:
        return x
    else:
        return 0

In [11]:
%timeit ufunc_zero_clamp(a_int16, 1600)
%timeit ufunc_zero_clamp(a_float32, 0.3)
%timeit ufunc_zero_clamp(a_float32_strided, 0.3)

15.4 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
14.1 µs ± 1.48 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
14.6 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Note that for this simple ufunc, Numba is not as fast as the function with the manual looping, and in some cases, is the same speed as the example that called NumPy directly.  This is not surprising as this function is very simple, and NumPy *also uses compiled ufuncs*.  Numba `@vectorize` is generally most effective when creating ufuncs that are not a simple combination of existing NumPy operations.

## Calling NumPy functions

Numba supports many, but not all, NumPy functions.  Some functions also have limitations that prevent the use of some of the optional arguments in nopython mode.  A full description can be found in the [Supported NumPy Features](http://numba.pydata.org/numba-doc/dev/reference/numpysupported.html) page in the Numba Reference Manual.

Note that when using NumPy functions on arrays, Numba will also compile and optimize array expressions:

In [12]:
def numpy_mpe(x, true):
    return (((x - true)/true)**2).mean()

numba_mpe = jit(nopython=True)(numpy_mpe)  # using jit as a function rather than a decorator

We can confirm both versions give the same answer:

In [13]:
true_x = 0.1
x = np.random.normal(true_x, 1, size=100000)
numpy_mpe(x, true=true_x), numba_mpe(x, true=true_x)

(99.79697067814203, 99.79697067814055)

And see the Numba version is faster:

In [14]:
%timeit numpy_mpe(x, true=0.1)
%timeit numba_mpe(x, true=0.1)

535 µs ± 9.91 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
302 µs ± 9.73 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
